In [17]:
import xarray as xr
import dask
import pandas as pd
import hvplot.xarray
import os

In [18]:
os.getcwd()

'/home/jovyan/nhw21-projects-upwelling/data'

In [4]:
from dask.distributed import Client
client = Client(n_workers=5, threads_per_worker=1)  # set up local cluster on your laptop
client

/srv/conda/envs/notebook/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39349 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:39349/status,
Dashboard: http://127.0.0.1:39349/status,Workers: 5
Total threads: 5,Total memory: 15.35 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34783,Workers: 5
Dashboard: http://127.0.0.1:39349/status,Total threads: 5
Started: Just now,Total memory: 15.35 GiB
Comm: tcp://127.0.0.1:42679,Total threads: 1
Dashboard: http://127.0.0.1:38665/status,Memory: 3.07 GiB
Nanny: tcp://127.0.0.1:38721,


In [5]:
http_links = open('../data/IOv2-links.txt').read().splitlines()[:365]
len(http_links)

365

In [6]:
%%time
d_cube = xr.concat([xr.open_dataset(l, chunks='auto') for l in http_links], dim='time')

CPU times: user 20.1 s, sys: 7.63 s, total: 27.7 s
Wall time: 3min 32s


In [7]:
d_cube

<xarray.Dataset>
Dimensions:           (lat: 720, lon: 1440, time: 365, nv: 2)
Coordinates:
  * lat               (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon               (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time              (time) datetime64[ns] 1981-09-01 1981-09-02 ... 1982-08-31
Dimensions without coordinates: nv
Data variables:
    lat_bnds          (time, lat, nv) float32 dask.array<chunksize=(1, 720, 2), meta=np.ndarray>
    lon_bnds          (time, lon, nv) float32 dask.array<chunksize=(1, 1440, 2), meta=np.ndarray>
    time_bnds         (time, nv) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/48)
    product_version:                 Version 2.0
    spatial_resolution:              0.25 degree
    Conventions:                     CF-1.6,ACDD-1.3
    title:                           NCEI global 0.25 deg daily sea surface t...
    references:                      Reynolds, et al.(2009) What is New in Ve...
    institution:                     NCEI
    ...                              ...
    source:                          AVHRR_Pathfinder-NODC-L3C-v5.1,ICOADS_SH...
    summary:                         NOAA's 1/4-degree Daily Optimum Interpol...
    time_coverage_start:             19810901T000000Z
    time_coverage_end:               19810902T000000Z
    uuid:                            39832cc3-d409-438a-820e-2bb1b38ebca8
    DODS_EXTRA.Unlimited_Dimension:  time

In [10]:
df = pd.read_csv('../data/sample_point_pairs_trim.csv')

In [12]:
# ras is the sstxr; pts is df
def getdf2(ras, pts, loc="ns"):
    ind_x = xr.DataArray(df["lon."+loc], dims=['i'])
    ind_y = xr.DataArray(df["lat."+loc], dims=['i'])
    xr_new = ras.analysed_sst.sel(lon=ind_x, lat=ind_y, method='nearest')
    return xr_new

In [13]:
sst_ns = getdf2(d_cube, df, loc="ns")
sst_os = getdf2(d_cube, df, loc="os")
sst_dif = sst_os - sst_ns

In [14]:
sst_dif

<xarray.DataArray 'analysed_sst' (time: 365, i: 567)>
dask.array<sub, shape=(365, 567), dtype=float32, chunksize=(1, 567), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1981-09-01 1981-09-02 ... 1982-08-31
  * i        (i) int64 0 1 2 3 4 5 6 7 8 ... 558 559 560 561 562 563 564 565 566

In [15]:
%%time
sst_dif_compute = sst_dif.compute()

CPU times: user 10.8 s, sys: 1.84 s, total: 12.6 s
Wall time: 1min 14s


In [16]:
sst_dif_compute.hvplot()

:Image   [i,time]   (analysed_sst)